In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import Dataset

In [ ]:
IMG_SIZE=224
BATCH_SIZE=32
CHANNELS=3

In [ ]:
IMAGE_SIZE=224
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=False
)

test_datagen = ImageDataGenerator(rescale = 1.0/255)

In [ ]:
train_data = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/Cotton Disease/train',
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

In [ ]:
train_data.class_indices

In [ ]:
test_data = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/Cotton Disease/test',
                                                  target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

## VGG16 Model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense
from glob import glob

Image_size = [224,224]

train_path = '/content/drive/MyDrive/Datasets/Cotton Disease/train'
test_path = '/content/drive/MyDrive/Datasets/Cotton Disease/test'

In [ ]:
vgg16 = VGG16(input_shape = Image_size + [3], weights = 'imagenet', include_top = False)

In [ ]:
vgg16.summary()

In [ ]:
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
number_of_classes = glob("/content/drive/MyDrive/Datasets/Cotton Disease/train/*")
number_of_classes


In [ ]:
from tensorflow.keras.layers import Flatten

flatten_layer = Flatten()(vgg16.output)
output_layer = Dense(len(number_of_classes),activation = 'softmax')(flatten_layer)

In [ ]:
from tensorflow.keras.models import Model
vgg16_model = Model(inputs = vgg16.input,outputs = output_layer)
vgg16_model.summary()

In [ ]:
vgg16_model.compile(loss= 'categorical_crossentropy',
                   optimizer = 'adam',
                   metrics = ['accuracy'])

In [ ]:
train_data = train_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/Cotton Disease/train',
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')
test_data = test_datagen.flow_from_directory('/content/drive/MyDrive/Datasets/Cotton Disease/test',
                                                    target_size = (224,224),
                                                    batch_size = 32,
                                                    class_mode = 'categorical')

In [ ]:
history = vgg16_model.fit_generator(train_data,validation_data= test_data,epochs = 10,
                                   steps_per_epoch=len(train_data),validation_steps=len(test_data))

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import os

test_folder = "/content/drive/MyDrive/Datasets/Cotton Disease/test"  # Replace with the path to your test folder
class_names = []

for class_label in os.listdir(test_folder):
    if os.path.isdir(os.path.join(test_folder, class_label)):
        class_names.append(class_label)

print("Found class_names:", class_names)


Predict Function

In [ ]:
scores = vgg16_model.evaluate(test_data)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [208]:
# Personal code for prediction. takes image path and predicts class and confidence

# from keras.preprocessing import image
# import numpy as np

# img_path = "/content/drive/MyDrive/Datasets/Cotton Disease/test/diseased cotton leaf/dis_leaf (260)_iaip.jpg"  # Replace with the path to your image file
# img = image.load_img(img_path, target_size=(224, 224))  # Adjust the target size to match your model's input size

# def predict(model, img):
#   img_array = image.img_to_array(img)

#   img_array = np.expand_dims(img_array, axis=0)

#   img_array = preprocess_input(img_array)

#   predictions = vgg16_model.predict(img_array)
#   confidence = round(100 * (np.max(predictions[0])), 2)
#   name = class_names[np.argmax(predictions)-1]

#   return confidence, name

In [239]:
from keras.preprocessing import image
import numpy as np
from PIL import Image

def predict(model, img):
  target_size = (224, 224)
  width_ratio = target_size[0] / img.width
  height_ratio = target_size[1] / img.height

  # Resize the image
  img = img.resize(target_size, Image.LANCZOS)

  # Convert the image to a numpy array
  img_array = tf.keras.preprocessing.image.img_to_array(img)

  img_array = np.expand_dims(img_array, axis=0)


  # Make predictions
  predictions = model.predict(img_array)

  # Get the label index with the highest confidence
  confidence = round(100 * (np.max(predictions[0])), 2)
  name = class_names[np.argmax(predictions)-1]

  return confidence, name

# Example usage:
image_path = "/content/drive/MyDrive/Datasets/Cotton Disease/test/diseased cotton leaf/dis_leaf (260)_iaip.jpg"
img = Image.open(image_path)

confidence, label_index = predict(vgg16_model, img)
print(f"Predicted Label Name: {label_index}")
print(f"Confidence: {confidence}")


1/1 [==============================] - 0s 43ms/step
Predicted Label Name: diseased cotton leaf
Confidence: 100.0


# Save Model-2

In [241]:
import os
model_version=max([int(i) for i in os.listdir("/content/drive/MyDrive/Datasets/models/") + [0]])+1
vgg16_model.save(f"/content/drive/MyDrive/Datasets/models/{model_version}")

In [242]:
vgg16_model.save("/content/drive/MyDrive/Datasets/models/new_v2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
